In [1]:
print(1)

1


# core

In [2]:
from pymich.test_utils import Compiler, VM, TestContract

def py_to_vm(py):
    contract = Compiler(py).compile_contract()
    vm = VM()
    vm.load_contract(contract)
    return vm

class InlineTestContract(TestContract):
    def setUp(self):
        source = self.get_source()
                
        self.micheline = Compiler(source).compile_contract()
        self.vm = VM()
        self.vm.load_contract(self.micheline)
        self.contract = self.vm.contract
        

# visitor

In [3]:
from pymich.michelson_types import Nat, Mutez, String, Address, BigMap, Record, Contract, Timestamp, Int
from pymich.stdlib import SENDER, AMOUNT

from dataclasses import dataclass


@dataclass
class VisitorInfo(Record):
    visits: Nat
    name: String
    last_visit: Timestamp

class VisitorContract(Contract):
    visitors: BigMap[Address, VisitorInfo]
    total_visits: Nat

    def register(self, name: String):
        self.visitors[SENDER] = VisitorInfo(Nat(0), name, Timestamp.now())

    def visit(self):
        if not (SENDER in self.visitors):
            raise Exception("You are not registered yet!")

        n_visits = self.visitors[SENDER].visits

        if Timestamp.now() - self.visitors[SENDER].last_visit < Int(10) * Int(24) * Int(3600):
            raise Exception("You need to wait 10 days between visits")

        if n_visits == Nat(0) and AMOUNT != Mutez(5):
            raise Exception("You need to pay 5 mutez on your first visit!")

        if n_visits != Nat(0) and AMOUNT != Mutez(3):
            raise Exception("You need to pay 3 mutez to visit!")

        self.visitors[SENDER].visits = n_visits + Nat(1)
        self.total_visits = self.total_visits + Nat(1)


In [4]:
visitor_source = In[next(i for i in reversed(range(len(In))) if 'class' + ' VisitorContract(' in In[i])]
visitor_source

'from pymich.michelson_types import Nat, Mutez, String, Address, BigMap, Record, Contract, Timestamp, Int\nfrom pymich.stdlib import SENDER, AMOUNT\n\nfrom dataclasses import dataclass\n\n\n@dataclass\nclass VisitorInfo(Record):\n    visits: Nat\n    name: String\n    last_visit: Timestamp\n\nclass VisitorContract(Contract):\n    visitors: BigMap[Address, VisitorInfo]\n    total_visits: Nat\n\n    def register(self, name: String):\n        self.visitors[SENDER] = VisitorInfo(Nat(0), name, Timestamp.now())\n\n    def visit(self):\n        if not (SENDER in self.visitors):\n            raise Exception("You are not registered yet!")\n\n        n_visits = self.visitors[SENDER].visits\n\n        if Timestamp.now() - self.visitors[SENDER].last_visit < Int(10) * Int(24) * Int(3600):\n            raise Exception("You need to wait 10 days between visits")\n\n        if n_visits == Nat(0) and AMOUNT != Mutez(5):\n            raise Exception("You need to pay 5 mutez on your first visit!")\n\n    

In [5]:
print(py_to_vm(visitor_source).contract.to_michelson())

parameter (or (string %register) (unit %visit));
storage (pair
          (map %visitors address (pair (nat %visits) (pair (string %name) (timestamp %last_visit))))
          (nat %total_visits));
code { UNPAIR ;
       DIG 0 ;
       IF_LEFT
         { NIL operation ;
           DUP 3 ;
           DUP ;
           GET 1 ;
           NOW ;
           DUP 5 ;
           PUSH nat 0 ;
           PAIR 3 ;
           SOME ;
           SENDER ;
           UPDATE ;
           UPDATE 1 ;
           DIP 3 { DROP } ;
           DUG 2 ;
           DUP 3 ;
           DUP 2 ;
           PAIR ;
           DIP 1 { DROP } ;
           DIP 1 { DROP } ;
           DIP 1 { DROP } }
         { NIL operation ;
           DUP 3 ;
           GET 1 ;
           SENDER ;
           MEM ;
           NOT ;
           IF { PUSH string "You are not registered yet!" ; FAILWITH } {} ;
           DUP 3 ;
           GET 1 ;
           SENDER ;
           GET ;
           IF_NONE { PUSH string "Key does not exist" ; FAI

In [6]:
import unittest

class VisitorTestContract(unittest.TestCase):
    def test_empty(self) -> None:
        self.assertEqual(1 == 0, 'Empty visitor test')

In [7]:
from datetime import datetime
from pymich.test_utils import TestContract

import pathlib
import os


class VisitorVmTestContract(InlineTestContract):
    def get_source(self):
        return visitor_source
    
    now = int(datetime.timestamp(datetime.now()))

    def test_register(self):
        init_storage = {'total_visits': 0, 'visitors': {}}

        user = self.make_test_account()
        new_storage = self.contract.register("Dogus").interpret(storage=init_storage, sender=user, now=self.now).storage

        self.assertEqual(new_storage['visitors'][user], {'visits': 0, 'name': 'Dogus', 'last_visit': self.now})
        self.assertEqual(new_storage['total_visits'], 0)

    def test_visit(self):
        user_1, user_2 = self.make_n_test_accounts(2)
        init_storage = {'total_visits': 0, 'visitors': {user_1: {'visits': 0, 'name': 'Dogus', 'last_visit': self.now}}}

        new_storage = self.contract.visit().interpret(storage=init_storage, sender=user_1, amount=5, now=self.now + 10 * 24 * 60 * 60).storage

        self.assertEqual(new_storage['visitors'][user_1]['visits'], 1)
        self.assertEqual(new_storage['total_visits'], 1)

        with self.raisesMichelsonError('You need to wait 10 days between visits'):
            self.contract.visit().interpret(storage=init_storage, sender=user_1, amount=5, now=self.now).storage

        with self.raisesMichelsonError('You need to pay 5 mutez on your first visit!'):
            self.contract.visit().interpret(storage=init_storage, sender=user_1, amount=0, now=self.now + 10 * 24 * 60 * 60).storage

        init_storage['visitors'][user_1]['visits'] = 1

        with self.raisesMichelsonError('You need to pay 3 mutez to visit!'):
            self.contract.visit().interpret(storage=init_storage, sender=user_1, amount=0, now=self.now + 10 * 24 * 60 * 60).storage

        with self.raisesMichelsonError('You are not registered yet!'):
            self.contract.visit().interpret(storage=init_storage, sender=user_2).storage


# lottery

In [8]:
from pymich.michelson_types import Nat, Mutez, Address, Bytes, BigMap, Record, Unit, Contract
from pymich.stdlib import SENDER, AMOUNT, BALANCE, transaction

from dataclasses import dataclass
from datetime import datetime


@dataclass
class BidInfo(Record):
    value_hash: Bytes
    num_bid: Nat


@dataclass
class LotteryContract(Contract):
    bid_amount: Mutez
    deadline_bet: datetime
    deadline_reveal: datetime
    bids: BigMap[Address, BidInfo]
    nb_bids: Nat
    nb_revealed: Nat
    sum_values: Nat

    def bet(self, value_hash: Bytes):
        if SENDER in self.bids:
            raise Exception("You have already made a bid")

        if AMOUNT != self.bid_amount:
            raise Exception("You have not bidded the right amount")

        if datetime.now() > self.deadline_bet:
            raise Exception("Too late to make a bid")

        self.bids[SENDER] = BidInfo(value_hash, self.nb_bids)
        self.nb_bids = self.nb_bids + Nat(1)

    def reveal(self, value: Nat):
        if not (SENDER in self.bids):
            raise Exception("You have not made a bid")

        if datetime.now() > self.deadline_bet or datetime.now() > self.deadline_reveal:
            raise Exception("Too late to make a reveal")

        if Bytes(value).blake2b() != self.bids[SENDER].value_hash:
            raise Exception("Wrong hash")

        self.sum_values = self.sum_values + value
        self.nb_revealed = self.nb_revealed + Nat(1)

    def claim(self):
        if datetime.now() < self.deadline_reveal:
            raise Exception("The lottery is not over")

        if not (SENDER in self.bids):
            raise Exception("You have not made a bid")

        num_winner = self.sum_values % self.nb_revealed

        if self.bids[SENDER].num_bid != num_winner:
            raise Exception("You have not won")

        transaction(Contract(SENDER), BALANCE, Unit)


In [9]:
lottery_source = In[next(i for i in reversed(range(len(In))) if 'class' + ' LotteryContract(' in In[i])]
lottery_source

'from pymich.michelson_types import Nat, Mutez, Address, Bytes, BigMap, Record, Unit, Contract\nfrom pymich.stdlib import SENDER, AMOUNT, BALANCE, transaction\n\nfrom dataclasses import dataclass\nfrom datetime import datetime\n\n\n@dataclass\nclass BidInfo(Record):\n    value_hash: Bytes\n    num_bid: Nat\n\n\n@dataclass\nclass LotteryContract(Contract):\n    bid_amount: Mutez\n    deadline_bet: datetime\n    deadline_reveal: datetime\n    bids: BigMap[Address, BidInfo]\n    nb_bids: Nat\n    nb_revealed: Nat\n    sum_values: Nat\n\n    def bet(self, value_hash: Bytes):\n        if SENDER in self.bids:\n            raise Exception("You have already made a bid")\n\n        if AMOUNT != self.bid_amount:\n            raise Exception("You have not bidded the right amount")\n\n        if datetime.now() > self.deadline_bet:\n            raise Exception("Too late to make a bid")\n\n        self.bids[SENDER] = BidInfo(value_hash, self.nb_bids)\n        self.nb_bids = self.nb_bids + Nat(1)\n\n

In [10]:
print(py_to_vm(lottery_source).contract.to_michelson())

parameter (or (or (bytes %bet) (unit %claim)) (nat %reveal));
storage (pair (mutez %bid_amount)
              (pair (timestamp %deadline_bet)
                    (pair (timestamp %deadline_reveal)
                          (pair (map %bids address (pair (bytes %value_hash) (nat %num_bid)))
                                (pair (nat %nb_bids) (pair (nat %nb_revealed) (nat %sum_values)))))));
code { UNPAIR ;
       DIG 0 ;
       IF_LEFT
         { IF_LEFT
             { NIL operation ;
               DUP 3 ;
               GET 7 ;
               SENDER ;
               MEM ;
               IF { PUSH string "You have already made a bid" ; FAILWITH } {} ;
               DUP 3 ;
               GET 1 ;
               AMOUNT ;
               COMPARE ;
               NEQ ;
               IF { PUSH string "You have not bidded the right amount" ; FAILWITH } {} ;
               DUP 3 ;
               GET 3 ;
               NOW ;
               COMPARE ;
               GT ;
               IF { PU

In [11]:
class LotteryTestContract(unittest.TestCase):
    def test_empty(self) -> None:
        self.assertEqual(1 == 0, 'Empty lottery test')

In [12]:
from pymich.test_utils import TestContract, VM, Compiler

from pytezos.crypto.key import blake2b_32

import pathlib
import os

def hash_value(value: int):
    vm = VM()
    source = f"Bytes(Nat({value})).blake2b()"
    micheline = Compiler(source).compile_expression()
    vm.execute(micheline)
    return vm.stack.peek().value



class LotteryVmTestContract(InlineTestContract):
    def get_source(self):
        return lottery_source
    
    def test_register(self):
        alice, bob = self.make_n_test_accounts(2)

        init_storage = {
            "bid_amount": 10,
            "deadline_bet": 10000,
            "deadline_reveal": 20000,
            "bids": {},
            "nb_bids": 0,
            "nb_revealed": 0,
            "sum_values": 0,
        }

        bob_value = 10
        bob_hash = blake2b_32(str(bob_value)).digest()
        alice_value = 15
        alice_hash = blake2b_32(str(alice_value)).digest()

        new_storage = self.contract.bet(alice_hash).interpret(storage=init_storage, sender=alice, amount=10).storage


        self.assertEqual(new_storage['bids'][alice], {"value_hash": alice_hash, "num_bid": 0})
        self.assertEqual(new_storage['nb_bids'], 1)

        with self.raisesMichelsonError('You have not bidded the right amount'):
            self.contract.bet(alice_hash).interpret(storage=init_storage, sender=alice, amount=5).storage

    def test_reveal(self):
        alice = self.make_test_account()

        alice_value = 15
        alice_hash = hash_value(alice_value)

        init_storage = {
            "bid_amount": 10,
            "deadline_bet": 10000,
            "deadline_reveal": 20000,
            "bids": {
                alice: {
                    "value_hash": alice_hash,
                    "num_bid": 1,
                }
            },
            "nb_bids": 0,
            "nb_revealed": 0,
            "sum_values": 0,
        }

        new_storage = self.contract.reveal(alice_value).interpret(storage=init_storage, sender=alice).storage
        self.assertEqual(new_storage['sum_values'], alice_value)
        self.assertEqual(new_storage['nb_revealed'], 1)

        with self.raisesMichelsonError('Wrong hash'):
            self.contract.reveal(alice_value + 10).interpret(storage=init_storage, sender=alice).storage

    def test_claim(self):
        alice, bob = self.make_n_test_accounts(2)

        alice_value = 15
        alice_hash = hash_value(alice_value)
        bob_value = 10
        bob_hash = hash_value(bob_value)

        init_storage = {
            "bid_amount": 10,
            "deadline_bet": 10000,
            "deadline_reveal": 20000,
            "bids": {
                alice: {
                    "value_hash": alice_hash,
                    "num_bid": 0,
                },
                bob: {
                    "value_hash": bob_hash,
                    "num_bid": 1,
                }
            },
            "nb_bids": 0,
            "nb_revealed": 2,
            "sum_values": alice_value + bob_value,
        }

        ops = self.contract.claim().interpret(storage=init_storage, sender=bob, now=1000000, balance=20).operations
        self.assertEqual(ops[0]["destination"], bob)
        self.assertEqual(ops[0]["amount"], "20")

        with self.raisesMichelsonError('You have not won'):
            self.contract.claim().interpret(storage=init_storage, sender=alice, now=1000000, balance=20).storage


# escrow

In [13]:
from pymich.michelson_types import Mutez, Address, Contract, Unit, Contract
from pymich.stdlib import SENDER, AMOUNT, transaction, BALANCE


class EscrowContract(Contract):
    seller: Address
    buyer: Address
    price: Mutez
    paid: bool
    confirmed: bool

    def pay(self):
        if SENDER != self.buyer:
            raise Exception("You are not the seller")

        if AMOUNT != self.price:
            raise Exception("Not the right price")

        if self.paid:
            raise Exception("You have already paid!")

        self.paid = True

    def confirm(self):
        if SENDER != self.buyer:
            raise Exception("You are not the buyer")

        if not self.paid:
            raise Exception("You have not paid")

        if self.confirmed:
            raise Exception("Already confirmed")

        self.confirmed = True

    def claim(self):
        if SENDER != self.seller:
            raise Exception("You are not the seller")

        if not self.confirmed:
            raise Exception("Not confirmed")

        transaction(Contract(SENDER), BALANCE, Unit)



In [14]:
escrow_source = In[next(i for i in reversed(range(len(In))) if 'class' + ' EscrowContract(' in In[i])]
escrow_source

'from pymich.michelson_types import Mutez, Address, Contract, Unit, Contract\nfrom pymich.stdlib import SENDER, AMOUNT, transaction, BALANCE\n\n\nclass EscrowContract(Contract):\n    seller: Address\n    buyer: Address\n    price: Mutez\n    paid: bool\n    confirmed: bool\n\n    def pay(self):\n        if SENDER != self.buyer:\n            raise Exception("You are not the seller")\n\n        if AMOUNT != self.price:\n            raise Exception("Not the right price")\n\n        if self.paid:\n            raise Exception("You have already paid!")\n\n        self.paid = True\n\n    def confirm(self):\n        if SENDER != self.buyer:\n            raise Exception("You are not the buyer")\n\n        if not self.paid:\n            raise Exception("You have not paid")\n\n        if self.confirmed:\n            raise Exception("Already confirmed")\n\n        self.confirmed = True\n\n    def claim(self):\n        if SENDER != self.seller:\n            raise Exception("You are not the seller")

In [15]:
print(py_to_vm(escrow_source).contract.to_michelson())

parameter (or (or (unit %claim) (unit %confirm)) (unit %pay));
storage (pair (address %seller)
              (pair (address %buyer) (pair (mutez %price) (pair (bool %paid) (bool %confirmed)))));
code { UNPAIR ;
       DIG 0 ;
       IF_LEFT
         { IF_LEFT
             { NIL operation ;
               DUP 3 ;
               GET 1 ;
               SENDER ;
               COMPARE ;
               NEQ ;
               IF { PUSH string "You are not the seller" ; FAILWITH } {} ;
               DUP 3 ;
               GET 8 ;
               NOT ;
               IF { PUSH string "Not confirmed" ; FAILWITH } {} ;
               DUP 1 ;
               SENDER ;
               CONTRACT unit ;
               IF_NONE { PUSH string "Contract does not exist" ; FAILWITH } {} ;
               BALANCE ;
               UNIT ;
               TRANSFER_TOKENS ;
               CONS ;
               DIP 1 { DROP } ;
               DUG 0 ;
               DUP 3 ;
               DUP 2 ;
               PAIR ;
 

In [16]:
class EscrowTestContract(unittest.TestCase):
    def test_empty(self) -> None:
        self.assertEqual(1 == 0, 'Empty escrow test')

In [17]:
from pymich.test_utils import TestContract

import pathlib
import os

class EscrowVmTestContract(InlineTestContract):
    def get_source(self):
        return escrow_source
    
    def test_pay(self):
        seller, buyer = self.make_n_test_accounts(2)
        price = 20

        init_storage = {
            'seller': seller,
            'buyer': buyer,
            'price': price,
            'paid': False,
            'confirmed': False,
        }

        new_storage = self.contract.pay().interpret(storage=init_storage, sender=buyer, amount=price).storage

        self.assertEqual(new_storage['paid'], True)

        with self.raisesMichelsonError('You have already paid!'):
            self.contract.pay().interpret(storage=new_storage, sender=buyer, amount=price).storage

        with self.raisesMichelsonError('Not the right price'):
            self.contract.pay().interpret(storage=init_storage, sender=buyer, amount=0).storage

        with self.raisesMichelsonError('You are not the seller'):
            self.contract.pay().interpret(storage=init_storage, sender=seller, amount=price).storage

    def test_confirm(self):
        seller, buyer = self.make_n_test_accounts(2)
        price = 20

        init_storage = {
            'seller': seller,
            'buyer': buyer,
            'price': price,
            'paid': True,
            'confirmed': False,
        }

        new_storage = self.contract.confirm().interpret(storage=init_storage, sender=buyer, amount=price).storage

        self.assertEqual(new_storage['confirmed'], True)

        with self.raisesMichelsonError('Already confirmed'):
            self.contract.confirm().interpret(storage=new_storage, sender=buyer, amount=price).storage

        with self.raisesMichelsonError('You are not the buyer'):
            self.contract.confirm().interpret(storage=init_storage, sender=seller, amount=price).storage

        init_storage['paid'] = False

        with self.raisesMichelsonError('You have not paid'):
            self.contract.confirm().interpret(storage=init_storage, sender=buyer, amount=price).storage

    def test_claim(self):
        seller, buyer = self.make_n_test_accounts(2)
        price = 20

        init_storage = {
            'seller': seller,
            'buyer': buyer,
            'price': price,
            'paid': True,
            'confirmed': True,
        }

        ops = self.contract.claim().interpret(storage=init_storage, sender=seller).operations

        self.assertEqual(len(ops), 1)
        self.assertEqual(ops[0]['destination'], seller)
        self.assertEqual(ops[0]['amount'], str(0))


# auction

In [18]:
from pymich.michelson_types import Mutez, Address, BigMap, Contract, Unit
from pymich.stdlib import SENDER, AMOUNT, transaction


class AuctionContract(Contract):
    owner: Address
    top_bidder: Address
    bids: BigMap[Address, Mutez]

    def bid(self):
        if SENDER in self.bids:
            raise Exception("You have already made a bid")

        self.bids[SENDER] = AMOUNT
        if AMOUNT > self.bids[self.top_bidder]:
            self.top_bidder = SENDER

    def collectTopBid(self):
        if SENDER != self.owner:
            raise Exception("Only the owner can collect the top bid")

        transaction(Contract(SENDER), self.bids[self.top_bidder], Unit)

    def claim(self):
        if not (SENDER in self.bids):
            raise Exception("You have not made any bids!")

        if SENDER == self.top_bidder:
            raise Exception("You won!")

        transaction(Contract(SENDER), self.bids[SENDER], Unit)


In [19]:
auction_source = In[next(i for i in reversed(range(len(In))) if 'class' + ' AuctionContract(' in In[i])]
auction_source

'from pymich.michelson_types import Mutez, Address, BigMap, Contract, Unit\nfrom pymich.stdlib import SENDER, AMOUNT, transaction\n\n\nclass AuctionContract(Contract):\n    owner: Address\n    top_bidder: Address\n    bids: BigMap[Address, Mutez]\n\n    def bid(self):\n        if SENDER in self.bids:\n            raise Exception("You have already made a bid")\n\n        self.bids[SENDER] = AMOUNT\n        if AMOUNT > self.bids[self.top_bidder]:\n            self.top_bidder = SENDER\n\n    def collectTopBid(self):\n        if SENDER != self.owner:\n            raise Exception("Only the owner can collect the top bid")\n\n        transaction(Contract(SENDER), self.bids[self.top_bidder], Unit)\n\n    def claim(self):\n        if not (SENDER in self.bids):\n            raise Exception("You have not made any bids!")\n\n        if SENDER == self.top_bidder:\n            raise Exception("You won!")\n\n        transaction(Contract(SENDER), self.bids[SENDER], Unit)'

In [20]:
print(py_to_vm(auction_source).contract.to_michelson())

parameter (or (or (unit %bid) (unit %claim)) (unit %collectTopBid));
storage (pair (address %owner) (pair (address %top_bidder) (map %bids address mutez)));
code { UNPAIR ;
       DIG 0 ;
       IF_LEFT
         { IF_LEFT
             { NIL operation ;
               DUP 3 ;
               GET 4 ;
               SENDER ;
               MEM ;
               IF { PUSH string "You have already made a bid" ; FAILWITH } {} ;
               DUP 3 ;
               DUP ;
               GET 4 ;
               AMOUNT ;
               SOME ;
               SENDER ;
               UPDATE ;
               UPDATE 4 ;
               DIP 3 { DROP } ;
               DUG 2 ;
               DUP 3 ;
               GET 4 ;
               DUP 4 ;
               GET 3 ;
               GET ;
               IF_NONE { PUSH string "Key does not exist" ; FAILWITH } {} ;
               AMOUNT ;
               COMPARE ;
               GT ;
               IF { DUP 3 ; SENDER ; UPDATE 3 ; DIP 3 { DROP } ; DUG 2 } {} 

In [21]:
class AuctionTestContract(unittest.TestCase):
    def test_empty(self) -> None:
        self.assertEqual(1 == 0, 'Empty auction test')

In [22]:
from pymich.test_utils import TestContract

import pathlib
import os

class AuctionVmTestContract(InlineTestContract):
    def get_source(self):
        return auction_source
    
    def test_register(self):
        admin, investor = self.make_n_test_accounts(2)

        init_storage = {'owner': admin, 'top_bidder': admin, 'bids': {admin: 0}}

        amount = 10
        new_storage = self.contract.bid().interpret(storage=init_storage, sender=investor, amount=amount).storage

        self.assertEqual(new_storage['top_bidder'], investor)
        self.assertEqual(new_storage['bids'][investor], amount)

        with self.raisesMichelsonError('You have already made a bid'):
            self.vm.contract.bid().interpret(storage=new_storage, sender=investor)

    def test_collect_top_bid(self):
        admin, investor = self.make_n_test_accounts(2)

        amount = 10
        init_storage = {'owner': admin, 'top_bidder': investor, 'bids': {investor: amount}}

        ops = self.contract.collectTopBid().interpret(storage=init_storage, sender=admin).operations

        self.assertEqual(len(ops), 1)
        self.assertEqual(ops[0]['destination'], admin)
        self.assertEqual(ops[0]['amount'], str(amount))

        with self.raisesMichelsonError('Only the owner can collect the top bid'):
            self.contract.collectTopBid().interpret(storage=init_storage, sender=investor)

    def test_claim(self):
        admin, investor_1, investor_2 = self.make_n_test_accounts(3)
        amount_1, amount_2 = 10, 5
        init_storage = {'owner': admin, 'top_bidder': investor_1, 'bids': {investor_1: amount_1, investor_2: amount_2}}

        ops = self.contract.claim().interpret(storage=init_storage, sender=investor_2).operations

        self.assertEqual(len(ops), 1)
        self.assertEqual(ops[0]['destination'], investor_2)
        self.assertEqual(ops[0]['amount'], str(amount_2))

        with self.raisesMichelsonError('You have not made any bids!'):
            self.contract.claim().interpret(storage=init_storage, sender=admin).operations

        with self.raisesMichelsonError('You won!'):
            self.contract.claim().interpret(storage=init_storage, sender=investor_1).operations


# election @

In [23]:
from dataclasses import dataclass
from pymich.michelson_types import Nat, String, Address, Contract
from pymich.stdlib import SENDER


def require(condition: bool, message: String) -> Nat:
    if not condition:
        raise Exception(message)

    return Nat(0)


@dataclass
class ElectionContract(Contract):
    admin: Address
    manifest_url: String
    manifest_hash: String
    _open: String
    _close: String
    artifacts_url: String
    artifacts_hash: String

    def open(self, _open: String, manifest_url: String, manifest_hash: String):
        require(SENDER == self.admin, String("Only admin can call this entrypoint 1"))
        self._open = _open
        self.manifest_url = manifest_url
        self.manifest_hash = manifest_hash

    def close(self, _close: String):
        require(SENDER == self.admin, String("Only admin can call this entrypoint 2"))
        self._close = _close

    def artifacts(self, artifacts_url: String, artifacts_hash: String):
        require(SENDER == self.admin, String("Only admin can call this entrypoint 3"))
        self.artifacts_url = artifacts_url
        self.artifacts_hash = artifacts_hash


In [24]:
election_source = In[next(i for i in reversed(range(len(In))) if 'class' + ' ElectionContract(' in In[i])]
election_source

'from dataclasses import dataclass\nfrom pymich.michelson_types import Nat, String, Address, Contract\nfrom pymich.stdlib import SENDER\n\n\ndef require(condition: bool, message: String) -> Nat:\n    if not condition:\n        raise Exception(message)\n\n    return Nat(0)\n\n\n@dataclass\nclass ElectionContract(Contract):\n    admin: Address\n    manifest_url: String\n    manifest_hash: String\n    _open: String\n    _close: String\n    artifacts_url: String\n    artifacts_hash: String\n\n    def open(self, _open: String, manifest_url: String, manifest_hash: String):\n        require(SENDER == self.admin, String("Only admin can call this entrypoint 1"))\n        self._open = _open\n        self.manifest_url = manifest_url\n        self.manifest_hash = manifest_hash\n\n    def close(self, _close: String):\n        require(SENDER == self.admin, String("Only admin can call this entrypoint 2"))\n        self._close = _close\n\n    def artifacts(self, artifacts_url: String, artifacts_hash: 

In [25]:
print(py_to_vm(election_source).contract.to_michelson())

parameter (or
            (or (pair %artifacts (string %artifacts_url) (string %artifacts_hash)) (string %close))
            (pair %open (string %_open) (pair (string %manifest_url) (string %manifest_hash))));
storage (pair (address %admin)
              (pair (string %manifest_url)
                    (pair (string %manifest_hash)
                          (pair (string %_open)
                                (pair (string %_close)
                                      (pair (string %artifacts_url) (string %artifacts_hash)))))));
code { UNPAIR ;
       DIG 1 ;
       IF_LEFT
         { IF_LEFT
             { NIL operation ;
               DUP 2 ;
               GET 1 ;
               DUP 3 ;
               GET 2 ;
               DIG 4 ;
               DUP ;
               DUG 5 ;
               PUSH string "Only admin can call this entrypoint 3" ;
               DUP 8 ;
               GET 1 ;
               SENDER ;
               COMPARE ;
               EQ ;
               PAIR 2 ;

In [26]:
import unittest
from pymich.michelson_types import String, Address
from pymich.stdlib import SENDER
#from tests.end_to_end.election.election import Election

#SENDER.address = "Mrs. Foo"

def get_contract(sender):
    return ElectionContract(sender, String(""), String(""), String(""), String(""), String(""), String(""))


class ElectionTestContract(unittest.TestCase):
    def test_open(self):
        contract = get_contract(SENDER)
        _open, _url, _hash = String("foo"), String("bar"), String("baz")
        contract.open(_open, _url, _hash)

        assert contract._open == _open
        assert contract.manifest_url == _url
        assert contract.manifest_hash == _hash

        contract = get_contract(Address("yolo"))
        try:
            contract.open(_open, _url, _hash)
            assert 0
        except Exception as e:
            assert e.args[0].value == 'Only admin can call this entrypoint 1'

    def test_close(self):
        contract = get_contract(SENDER)
        close = String("foo")
        contract.close(close)

        assert contract._close == close

        contract = get_contract(Address("yolo"))
        try:
            contract.close(String("foo"))
            assert 0
        except Exception as e:
            assert e.args[0].value == 'Only admin can call this entrypoint 2'

    def test_artifacts(self):
        contract = get_contract(SENDER)
        url, hash = String("url"), String("hash")
        contract.artifacts(url, hash)

        assert contract.artifacts_url == url
        assert contract.artifacts_hash == hash

        contract = get_contract(Address("yolo"))
        try:
            contract.artifacts(url, hash)
            assert 0
        except Exception as e:
            assert e.args[0].value == 'Only admin can call this entrypoint 3'


In [27]:
from pymich.test_utils import TestContract

import pathlib
import os

class ElectionVmTestContract(InlineTestContract):
    def get_source(self):
        return election_source

    def test_open(self):
        init_storage = self.contract.storage.dummy()
        admin = self.make_test_account()
        init_storage['admin'] = admin

        new_storage = self.contract.open({"_open": "foo", "manifest_url": "bar", "manifest_hash": "baz"}).interpret(storage=init_storage, sender=admin).storage
        self.assertEqual(new_storage['_open'], "foo")
        self.assertEqual(new_storage['manifest_url'], "bar")
        self.assertEqual(new_storage['manifest_hash'], "baz")

        with self.raisesMichelsonError('Only admin can call this entrypoint'):
            new_storage = self.contract.open({"_open": "foo", "manifest_url": "bar", "manifest_hash": "baz"}).interpret(storage=init_storage)

    def test_close(self):
        init_storage = self.contract.storage.dummy()
        admin = self.make_test_account()
        init_storage['admin'] = admin

        new_storage = self.contract.close("foo").interpret(storage=init_storage, sender=admin).storage
        self.assertEqual(new_storage['_close'], "foo")

        with self.raisesMichelsonError('Only admin can call this entrypoint'):
            new_storage = self.contract.close("foo").interpret(storage=init_storage)

    def test_artifacts(self):
        init_storage = self.contract.storage.dummy()
        admin = self.make_test_account()
        init_storage['admin'] = admin

        new_storage = self.contract.artifacts({"artifacts_url": "url", "artifacts_hash": "hash"}).interpret(storage=init_storage, sender=admin).storage
        self.assertEqual(new_storage['artifacts_url'], "url")
        self.assertEqual(new_storage['artifacts_hash'], "hash")

        with self.raisesMichelsonError('Only admin can call this entrypoint'):
            new_storage = self.contract.close("foo").interpret(storage=init_storage)


# fa2 @

In [28]:
from dataclasses import dataclass
from pymich.michelson_types import Nat, String, Address, Bytes, BigMap, List, Record
from pymich.stdlib import SENDER


@dataclass#(eq=True, frozen=True)
class OperatorKey(Record):
    owner: Address
    operator: Address
    token_id: Nat

@dataclass#(eq=True, frozen=True)
class LedgerKey(Record):
    owner: Address
    token_id: Nat

@dataclass#(eq=True, frozen=True)
class TokenMetadata(Record):
    token_id: Nat
    token_info: BigMap[String, Bytes]

@dataclass
class TransactionInfo(Record):
    to_: Address
    token_id: Nat
    amount: Nat

@dataclass
class TransferArgs(Record):
    from_: Address
    txs: List[TransactionInfo]


def require_owner(owner: Address) -> Nat:
    if SENDER != owner:
        raise Exception("FA2_NOT_CONTRACT_ADMINISTRATOR")

    return Nat(0)


@dataclass
class FA2Contract(Contract):
    ledger: BigMap[LedgerKey, Nat]
    operators: BigMap[OperatorKey, Nat]
    token_total_supply: BigMap[Nat, Nat]
    token_metadata: BigMap[Nat, TokenMetadata]
    owner: Address

    def create_token(self, metadata: TokenMetadata):
        require_owner(self.owner)

        new_token_id = metadata.token_id

        if new_token_id in self.token_metadata:
            raise Exception("FA2_DUP_TOKEN_ID")
        else:
            self.token_metadata[new_token_id] = metadata
            self.token_total_supply[new_token_id] = Nat(0)

    def mint_tokens(self, owner: Address, token_id: Nat, amount: Nat):
        require_owner(self.owner)

        if not token_id in self.token_metadata:
            raise Exception("FA2_TOKEN_DOES_NOT_EXIST")

        ledger_key = LedgerKey(owner, token_id)
        self.ledger[ledger_key] = self.ledger.get(ledger_key, Nat(0)) + amount
        self.token_total_supply[token_id] = self.token_total_supply[token_id] + amount

    def transfer(self, transactions: List[TransferArgs]):
        for transaction in transactions:
            for tx in transaction.txs:
                if not tx.token_id in self.token_metadata:
                    raise Exception("FA2_TOKEN_UNDEFINED")
                else:
                    if not (transaction.from_ == SENDER or OperatorKey(transaction.from_, SENDER, tx.token_id) in self.operators):
                        raise Exception("FA2_NOT_OPERATOR")

                    from_key = LedgerKey(transaction.from_, tx.token_id)
                    from_balance = self.ledger.get(from_key, Nat(0))

                    if tx.amount > from_balance:
                        raise Exception("FA2_INSUFFICIENT_BALANCE")

                    self.ledger[from_key] = abs(from_balance - tx.amount)

                    to_key = LedgerKey(tx.to_, tx.token_id)
                    self.ledger[to_key] = self.ledger.get(to_key, Nat(0)) + tx.amount

    def updateOperator(self, owner: Address, operator: Address, token_id: Nat, add_operator: bool):
        if SENDER != owner:
            raise Exception("FA2_NOT_OWNER")

        operator_key = OperatorKey(owner, operator, token_id)
        if add_operator:
            self.operators[operator_key] = Nat(0)
        #else:
        #    del self.operators[operator_key]

    def balanceOf(self, owner: Address, token_id: Nat) -> Nat:
        if not token_id in self.token_metadata:
            raise Exception("FA2_TOKEN_UNDEFINED")

        return self.ledger.get(LedgerKey(owner, token_id), Nat(0))


In [29]:
fa2_source = In[next(i for i in reversed(range(len(In))) if 'class' + ' FA2Contract(' in In[i])]
fa2_source

'from dataclasses import dataclass\nfrom pymich.michelson_types import Nat, String, Address, Bytes, BigMap, List, Record\nfrom pymich.stdlib import SENDER\n\n\n@dataclass#(eq=True, frozen=True)\nclass OperatorKey(Record):\n    owner: Address\n    operator: Address\n    token_id: Nat\n\n@dataclass#(eq=True, frozen=True)\nclass LedgerKey(Record):\n    owner: Address\n    token_id: Nat\n\n@dataclass#(eq=True, frozen=True)\nclass TokenMetadata(Record):\n    token_id: Nat\n    token_info: BigMap[String, Bytes]\n\n@dataclass\nclass TransactionInfo(Record):\n    to_: Address\n    token_id: Nat\n    amount: Nat\n\n@dataclass\nclass TransferArgs(Record):\n    from_: Address\n    txs: List[TransactionInfo]\n\n\ndef require_owner(owner: Address) -> Nat:\n    if SENDER != owner:\n        raise Exception("FA2_NOT_CONTRACT_ADMINISTRATOR")\n\n    return Nat(0)\n\n\n@dataclass\nclass FA2Contract(Contract):\n    ledger: BigMap[LedgerKey, Nat]\n    operators: BigMap[OperatorKey, Nat]\n    token_total_su

In [30]:
print(py_to_vm(fa2_source).contract.to_michelson())

parameter (or
            (or
              (or (pair %balanceOf (address %owner) (pair (nat %token_id) (contract nat)))
                  (pair %create_token (nat %token_id) (map %token_info string bytes)))
              (or (pair %mint_tokens (address %owner) (pair (nat %token_id) (nat %amount)))
                  (list %transfer (pair (address %from_)
                                       (list (pair (address %to_)
                                                  (pair (nat %token_id) (nat %amount))))))))
            (pair %updateOperator (address %owner)
                                  (pair (address %operator)
                                        (pair (nat %token_id) (bool %add_operator)))));
storage (pair (map %ledger (pair (address %owner) (nat %token_id)) nat)
              (pair
                (map %operators (pair (address %owner) (pair (address %operator) (nat %token_id)))
                                nat)
                (pair (map %token_total_supply nat nat)
 

In [31]:
class FA2TestContract(unittest.TestCase):
    def test_empty(self) -> None:
        self.assertEqual(1 == 0, 'Empty fa2 test')

In [32]:
from pymich.test_utils import TestContract

import pathlib
import os

class FA2VmTestContract(InlineTestContract):
    def get_source(self):
        return fa2_source
    
    admin = "tz3M4KAnKF2dCSjqfa1LdweNxBGQRqzvPL88"
    investor = "KT1EwUrkbmGxjiRvmEAa8HLGhjJeRocqVTFi"

    def test_create_token(self):
        storage = self.contract.storage.dummy()
        owner = self.make_test_account()
        storage["owner"] = owner
        new_storage = self.contract.create_token({"token_id": 1, "token_info": {"yo": bytes("hello my name is".encode("latin-1"))}}).interpret(storage=storage, sender=owner).storage
        self.assertEqual(new_storage["token_metadata"][1]["token_info"]["yo"], b"hello my name is")

        with self.raisesMichelsonError('FA2_DUP_TOKEN_ID'):
            self.contract.create_token({"token_id": 1, "token_info": {"yo": bytes("hello my name is".encode("latin-1"))}}).interpret(storage=new_storage, sender=owner).storage

    def test_mint(self):
        storage = self.contract.storage.dummy()
        owner = self.make_test_account()
        storage["owner"] = owner
        new_storage = self.contract.create_token({"token_id": 1, "token_info": {"yo": bytes("hello my name is".encode("latin-1"))}}).interpret(storage=storage, sender=owner).storage
        self.assertEqual(new_storage["token_metadata"][1]["token_info"]["yo"], b"hello my name is")
        new_storage = self.contract.mint_tokens({"owner": owner, "token_id": 1, "amount": 10}).interpret(storage=new_storage, sender=owner).storage

        self.assertEqual(new_storage["token_total_supply"], {1: 10})
        self.assertEqual(new_storage["ledger"][(owner, 1)], 10)

    def test_mint_two_tokens(self):
        storage = self.contract.storage.dummy()
        owner = self.make_test_account()
        storage["owner"] = owner
        new_storage = self.contract.create_token({"token_id": 1, "token_info": {"yo": bytes("hello my name is".encode("latin-1"))}}).interpret(storage=storage, sender=owner).storage
        new_storage = self.contract.mint_tokens({"owner": owner, "token_id": 1, "amount": 10}).interpret(storage=new_storage, sender=owner).storage
        new_storage = self.contract.create_token({"token_id": 2, "token_info": {"yo": bytes("hello my name is".encode("latin-1"))}}).interpret(storage=new_storage, sender=owner).storage
        new_storage = self.contract.mint_tokens({"owner": owner, "token_id": 2, "amount": 10}).interpret(storage=new_storage, sender=owner).storage

    def test_transfer(self):
        storage = self.contract.storage.dummy()
        owner, investor = self.make_n_test_accounts(2)
        storage["owner"] = owner
        new_storage = self.contract.create_token({"token_id": 1, "token_info": {"yo": bytes("hello my name is".encode("latin-1"))}}).interpret(storage=storage, sender=owner).storage

        investor = "KT1EwUrkbmGxjiRvmEAa8HLGhjJeRocqVTFi"
        new_storage = self.contract.mint_tokens({"owner": owner, "token_id": 1, "amount": 10}).interpret(storage=new_storage, sender=owner).storage

        transfer_param = [
            {
                "from_": owner,
                "list_1": [
                    {"amount": 4, "token_id": 1, "to_": investor},
                    {"amount": 4, "token_id": 1, "to_": investor},
                ]
            }
        ]
        new_storage = self.contract.transfer(transfer_param).interpret(storage=new_storage, sender=owner).storage

        self.assertEqual(new_storage["ledger"][(owner, 1)], 2)
        self.assertEqual(new_storage["ledger"][(investor, 1)], 8)

        new_storage["operators"] = {
            (investor, owner, 1): 0,
        }
        transfer_param = [
            {
                "from_": owner,
                "list_1": [
                    {"amount": 1, "token_id": 1, "to_": investor},
                    {"amount": 1, "token_id": 1, "to_": investor},
                ]
            },
            {
                "from_": investor,
                "list_1": [
                    {"amount": 2, "token_id": 1, "to_": owner},
                    {"amount": 2, "token_id": 1, "to_": owner},
                ]
            }
        ]
        new_storage = self.contract.transfer(transfer_param).interpret(storage=new_storage, sender=owner).storage
        self.assertEqual(new_storage["ledger"][(owner, 1)], 4)
        self.assertEqual(new_storage["ledger"][(investor, 1)], 6)

        new_storage = self.contract.create_token({"token_id": 2, "token_info": {"lo": bytes("slim shady".encode("latin-1"))}}).interpret(storage=new_storage, sender=owner).storage

        new_storage = self.contract.mint_tokens({"owner": owner, "token_id": 2, "amount": 10}).interpret(storage=new_storage, sender=owner).storage

        new_storage["operators"][(investor, owner, 2)] = 0

        transfer_param = [
            {
                "from_": owner,
                "list_1": [
                    {"amount": 2, "token_id": 2, "to_": investor},
                    {"amount": 2, "token_id": 2, "to_": investor},
                ]
            },
            {
                "from_": investor,
                "list_1": [
                    {"amount": 1, "token_id": 1, "to_": owner},
                    {"amount": 1, "token_id": 1, "to_": owner},
                ]
            }
        ]

        new_storage = self.contract.transfer(transfer_param).interpret(storage=new_storage, sender=owner).storage


# fa12

In [33]:
from dataclasses import dataclass
from pymich.michelson_types import Nat, Address, Int, Record, BigMap, Contract
from pymich.stdlib import SENDER


@dataclass(eq=True, frozen=True)
class AllowanceKey(Record):
    owner: Address
    spender: Address


@dataclass
class FA12Contract(Contract):
    tokens: BigMap[Address, Nat]
    allowances: BigMap[AllowanceKey, Nat]
    total_supply: Nat
    owner: Address

    def mint(self, _to: Address, value: Nat):
        if SENDER != self.owner:
            raise Exception("Only owner can mint")

        self.total_supply = self.total_supply + value

        self.tokens[_to] = self.tokens.get(_to, Nat(0)) + value

    def approve(self, spender: Address, value: Nat):
        allowance_key = AllowanceKey(SENDER, spender)

        previous_value = self.allowances.get(allowance_key, Nat(0))

        if previous_value > Nat(0) and value > Nat(0):
            raise Exception("UnsafeAllowanceChange")

        self.allowances[allowance_key] = value

    def transfer(self, _from: Address, _to: Address, value: Nat):
        if SENDER != _from:
            allowance_key = AllowanceKey(_from, SENDER)

            authorized_value = self.allowances.get(allowance_key, Nat(0))

            if (authorized_value - value) < Int(0):
                raise Exception("NotEnoughAllowance")

            self.allowances[allowance_key] = abs(authorized_value - value)

        from_balance = self.tokens.get(_from, Nat(0))

        if (from_balance - value) < Int(0):
            raise Exception("NotEnoughBalance")

        self.tokens[_from] = abs(from_balance - value)

        to_balance = self.tokens.get(_to, Nat(0))

        self.tokens[_to] = to_balance + value

    def getAllowance(self, owner: Address, spender: Address) -> Nat:
        return self.allowances.get(AllowanceKey(owner, spender), Nat(0))

    def getBalance(self, owner: Address) -> Nat:
        return self.tokens.get(owner, Nat(0))

    def getTotalSupply(self) -> Nat:
        return self.total_supply


In [34]:
fa12_source = In[next(i for i in reversed(range(len(In))) if 'class' + ' FA12Contract(' in In[i])]
fa12_source

'from dataclasses import dataclass\nfrom pymich.michelson_types import Nat, Address, Int, Record, BigMap, Contract\nfrom pymich.stdlib import SENDER\n\n\n@dataclass(eq=True, frozen=True)\nclass AllowanceKey(Record):\n    owner: Address\n    spender: Address\n\n\n@dataclass\nclass FA12Contract(Contract):\n    tokens: BigMap[Address, Nat]\n    allowances: BigMap[AllowanceKey, Nat]\n    total_supply: Nat\n    owner: Address\n\n    def mint(self, _to: Address, value: Nat):\n        if SENDER != self.owner:\n            raise Exception("Only owner can mint")\n\n        self.total_supply = self.total_supply + value\n\n        self.tokens[_to] = self.tokens.get(_to, Nat(0)) + value\n\n    def approve(self, spender: Address, value: Nat):\n        allowance_key = AllowanceKey(SENDER, spender)\n\n        previous_value = self.allowances.get(allowance_key, Nat(0))\n\n        if previous_value > Nat(0) and value > Nat(0):\n            raise Exception("UnsafeAllowanceChange")\n\n        self.allowa

In [35]:
print(py_to_vm(fa12_source).contract.to_michelson())

parameter (or
            (or
              (or (pair %approve (address %spender) (nat %value))
                  (pair %getAllowance (address %owner) (pair (address %spender) (contract nat))))
              (or (pair %getBalance (address %owner) (contract nat)) (contract %getTotalSupply nat)))
            (or (pair %mint (address %_to) (nat %value))
                (pair %transfer (address %_from) (pair (address %_to) (nat %value)))));
storage (pair (map %tokens address nat)
              (pair (map %allowances (pair (address %owner) (address %spender)) nat)
                    (pair (nat %total_supply) (address %owner))));
code { UNPAIR ;
       DIG 0 ;
       IF_LEFT
         { IF_LEFT
             { IF_LEFT
                 { NIL operation ;
                   DUP 2 ;
                   GET 1 ;
                   DUP 3 ;
                   GET 2 ;
                   DUP 2 ;
                   SENDER ;
                   PAIR 2 ;
                   DUP 6 ;
                   GET 3 ;

In [36]:
import unittest
from pymich.stdlib import SENDER
from pymich.michelson_types import Nat, Address, BigMap, KeyType, ValueType

admin = Address("tz3M4KAnKF2dCSjqfa1LdweNxBGQRqzvPL88")
investor = Address("KT1EwUrkbmGxjiRvmEAa8HLGhjJeRocqVTFi")

from typing import Dict


def dict_to_big_map(d: Dict[KeyType, ValueType]) -> BigMap[KeyType, ValueType]:
    big_map = BigMap[KeyType, ValueType]()
    for key, value in d.items():
        big_map[key] = value
    return big_map


def FactoryFA12(owner: Address, tokens_dict: Dict[Address, Nat]) -> FA12Contract:
    tokens = BigMap[Address, Nat]()
    total_supply = Nat(0)
    for address, amount in tokens_dict.items():
        tokens[address] = amount
        total_supply += amount
    return FA12Contract(owner=owner, tokens=tokens, allowances=BigMap(), total_supply=total_supply)


class FA12TestContract(unittest.TestCase):
    def test_mint(self) -> None:
        contract = FactoryFA12(admin, {})
        amount = Nat(10)
        SENDER.address = admin.address
        contract.mint(admin, amount)

        assert contract.tokens[admin] == amount

        contract = FA12Contract(owner=investor, tokens=BigMap(), allowances=BigMap(), total_supply=Nat(0))
        with self.assertRaises(Exception) as e:
            contract.mint(investor, amount)
        self.assertEqual(e.exception.args[0], 'Only owner can mint')

    def test_transfer(self) -> None:
        amount_1, amount_2 = Nat(10), Nat(4)
        contract = FactoryFA12(admin, {admin: amount_1})

        contract.transfer(admin, investor, amount_2)

        assert contract.tokens[admin] == abs(amount_1 - amount_2)
        assert contract.tokens[investor] == amount_2

        with self.assertRaises(Exception) as e:
            contract.transfer(admin, investor, Nat(100))
        self.assertEqual(e.exception.args[0], 'NotEnoughBalance')

    def todo_test_approve(self) -> None:
        # make dataclasses hashable by default by ignoring `unsafe_hash` in dataclass param in compiler)
        amount = Nat(10)
        contract = FactoryFA12(admin, {})
        contract.approve(investor, amount)
        breakpoint()
        assert contract.allowances[AllowanceKey(admin, investor)] == amount

In [37]:
import pathlib
import os

from pymich.test_utils import TestContract


class FA12VmTestContract(InlineTestContract):
    def get_source(self):
        return fa12_source

    def test_mint(self):
        storage = self.contract.storage.dummy()
        owner = self.make_test_account()
        storage['owner'] = owner

        storage = self.contract.mint({"_to": owner, "value": 10}).interpret(storage=storage, sender=owner).storage
        self.assertEqual(storage['tokens'], {owner: 10})

        with self.raisesMichelsonError('Only owner can mint'):
            self.contract.mint({"_to": owner, "value": 10}).interpret(storage=storage).storage

    def test_getAllowance(self):
        storage = self.contract.storage.dummy()
        owner, investor = self.make_n_test_accounts(2)
        storage['owner'] = owner

        amount = 10
        initial_storage = {"owner": owner, "total_supply": amount, "tokens": {}, "allowances": {(owner, investor): amount}}
        res = self.contract.getAllowance({"owner": owner, "spender": investor, "contract_2": None}).callback_view(storage=initial_storage)
        self.assertEqual(res, 10)

        res = self.contract.getAllowance({"owner": owner, "spender": investor, "contract_2": None}).callback_view()
        self.assertEqual(res, 0)

    def test_getBalance(self):
        storage = self.contract.storage.dummy()
        owner, investor = self.make_n_test_accounts(2)
        storage['owner'] = owner

        amount = 10
        initial_storage = {"owner": owner, "total_supply": amount, "tokens": {investor: amount}, "allowances": {}}
        res = self.contract.getBalance({"owner": investor, "contract_1": None}).callback_view(storage=initial_storage)
        self.assertEqual(res, 10)

        res = self.contract.getBalance({"owner": owner, "contract_1": None}).callback_view()
        self.assertEqual(res, 0)

    def test_getTotalSupply(self):
        self.assertEqual(self.contract.getTotalSupply().callback_view(), 0)

    def test_transfer(self):
        storage = self.contract.storage.dummy()
        owner, investor = self.make_n_test_accounts(2)
        storage['owner'] = owner
        storage['tokens'] = {owner: 10}

        storage = self.contract.transfer({"_to": investor, "_from": owner, "value": 4}).interpret(storage=storage, sender=owner).storage
        self.assertEqual(storage['tokens'], {owner: 6, investor: 4})

        with self.raisesMichelsonError('NotEnoughBalance'):
            self.contract.transfer({"_from": owner, "_to": investor, "value": 10}).interpret(storage=storage, sender=owner).storage

        with self.raisesMichelsonError('NotEnoughAllowance'):
            self.contract.transfer({"_from": owner, "_to": investor, "value": 10}).interpret(storage=storage, sender=investor).storage

        storage["allowances"] = {(owner, investor): 10}
        storage = self.contract.transfer({"_from": owner, "_to": investor, "value": 2}).interpret(storage=storage, sender=investor).storage
        assert storage["tokens"][investor] == 6

        with self.raisesMichelsonError('NotEnoughBalance'):
            self.contract.transfer({"_from": owner, "_to": investor, "value": 8}).interpret(storage=storage, sender=investor).storage

    def test_approve(self):
        storage = self.contract.storage.dummy()
        owner, investor = self.make_n_test_accounts(2)
        storage['owner'] = owner

        storage = self.contract.approve({"spender": investor, "value": 4}).interpret(storage=storage, sender=owner).storage
        self.assertEqual(storage['allowances'], {(owner, investor): 4})


In [38]:
import unittest
unittest.main(argv=[''], verbosity=2, exit=False)

test_empty (__main__.AuctionTestContract) ... FAIL
test_claim (__main__.AuctionVmTestContract) ... ok
test_collect_top_bid (__main__.AuctionVmTestContract) ... ok
test_register (__main__.AuctionVmTestContract) ... ok
test_artifacts (__main__.ElectionTestContract) ... ok
test_close (__main__.ElectionTestContract) ... ok
test_open (__main__.ElectionTestContract) ... ok
test_artifacts (__main__.ElectionVmTestContract) ... ERROR
test_close (__main__.ElectionVmTestContract) ... ERROR
test_open (__main__.ElectionVmTestContract) ... ERROR
test_empty (__main__.EscrowTestContract) ... FAIL
test_claim (__main__.EscrowVmTestContract) ... ok
test_confirm (__main__.EscrowVmTestContract) ... ok
test_pay (__main__.EscrowVmTestContract) ... ok
test_mint (__main__.FA12TestContract) ... ok
test_transfer (__main__.FA12TestContract) ... ok
test_approve (__main__.FA12VmTestContract) ... ok
test_getAllowance (__main__.FA12VmTestContract) ... ERROR
test_getBalance (__main__.FA12VmTestContract) ... ERROR
test

ERROR: test_mint (__main__.FA2VmTestContract)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/pytezos/michelson/micheline.py", line 27, in wrapper
    return func(*args, **kwargs)
  File "/pytezos/michelson/micheline.py", line 227, in assert_type_in
    assert any(issubclass(cls, ty) for ty in others), f'expected one of {expected}, got {cls.prim}{comment}'
AssertionError: expected one of ['or'], got pair

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/pytezos/michelson/micheline.py", line 27, in wrapper
    return func(*args, **kwargs)
  File "/pytezos/michelson/instructions/control.py", line 160, in execute
    or_.assert_type_in(OrType)
  File "/pytezos/michelson/micheline.py", line 33, in wrapper
    raise MichelsonRuntimeError(*e.args) from e
pytezos.michelson.micheline.MichelsonRuntimeError: ('IF_LEFT', "expected one of ['or'], got pair")

The above

FAIL: test_empty (__main__.AuctionTestContract)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/run/user/1000/ipykernel_626238/3922802555.py", line 3, in test_empty
    self.assertEqual(1 == 0, 'Empty auction test')
AssertionError: False != 'Empty auction test'

FAIL: test_empty (__main__.EscrowTestContract)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/run/user/1000/ipykernel_626238/3506637934.py", line 3, in test_empty
    self.assertEqual(1 == 0, 'Empty escrow test')
AssertionError: False != 'Empty escrow test'

FAIL: test_empty (__main__.FA2TestContract)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/run/user/1000/ipykernel_626238/296850332.py", line 3, in test_empty
    self.assertEqual(1 == 0, 'Empty fa2 test')
AssertionError: False != 'Empty fa2 test'

FAIL: test_empty (__main__